# Import Library

In [23]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import Reader, Dataset
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import GridSearchCV

# Import Data User, Anime, UserAnime

In [2]:
folder = "../Dataset SKRIPSI/"

filename = folder + "users_data"
UsersDF = pickle.load(open(filename,'rb'))

filename = folder + "animes_data"
AnimesDF = pickle.load(open(filename,'rb'))

filename = folder + "scores_data"
ScoresDF = pickle.load(open(filename,'rb'))

# Mengkalikan Matriks User-Item dengan Status

Jika Rating[i] dan Status[i] tidak ada
maka Rating[i] * 2

In [3]:
ScoresDF

,username,anime_id,my_score,my_status,animes_rated
0,Tsundora,21,2,4,259
1,Tsundora,59,5,2,259
2,Tsundora,210,5,2,259
3,Tsundora,249,7,4,259
4,Tsundora,269,6,2,259
...,...,...,...,...,...
193267,Senpai_Helix,4214,10,2,4
193268,Senpai_Helix,18677,9,2,4
193269,Kiyoshi-Kun,1722,10,2,3
193270,Kiyoshi-Kun,2993,10,2,3


# Membuat User-Item Matriks Berdasarkan Rating
Serta mengisi data yang kosong dengan 0

In [6]:
UserItemRatingMatrix = ScoresDF.pivot(index='username',columns='anime_id',values='my_score')
UserItemRatingMatrix = UserItemRatingMatrix.replace(np.nan,0)
UserItemRatingMatrix

anime_id,1,5,6,7,8,15,16,17,18,19,...,37277,37283,37320,37324,37341,37408,37433,37505,37672,37719
username,,,,,,,,,,,,,,,,,,,,,
-Axel-,10.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Belatriz-,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-INFERNO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Jhon-,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Sato-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zerorider,10.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ziomalos,9.0,8.0,9.0,7.0,0.0,8.0,9.0,8.0,8.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zipyto,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Training dan Test Model
Training dan test model dipisah 80% : 20%

In [8]:
reader = Reader(rating_scale=(1,10))
data = Dataset.load_from_df(ScoresDF[['username','anime_id','my_score']],reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=50)

# Tuning Parameter

In [19]:
svd_param = {
    'n_epochs' : [5,10,15,20,25],
    'lr_all' : [0.002, 0.005, 0.01],
    'reg_all' :[0.4, 0.6, 0.8]
}

knn_param = {
    'k' : [5,10,15,20,25],
    'sim_options':{
        'name': ['cosine', 'pearson_baseline','pearson'],
        'min_support' : [1,5]
    },
    'bsl_options': {
        'method': ['als', 'sgd'],
        'reg': [1, 2]
    }
}

gsSVD = GridSearchCV(SVD, svd_param, measures=['rmse','mae'], cv=3)
gsSVD.fit(data)

gsKNN = GridSearchCV(KNNBaseline, knn_param, measures=['rmse','mae'], cv=3)
gsKNN.fit(data)

print("Best Score SVD : ", gsSVD.best_score['rmse'])
print("Best Param SVD : ", gsSVD.best_params['rmse'])

print("Best Score KNNBaseline : ", gsKNN.best_score['rmse'])
print("Best Param KNNBaseline : ", gsKNN.best_params['rmse'])

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Comput

# SVD

In [22]:
algo_svd = SVD(n_epochs=25, lr_all=0.01, reg_all=0.04)
prediction_mf = algo_svd.fit(trainset).test(testset)

prediction_mf

[Prediction(uid='Shouichirou', iid=12679, r_ui=8.0, est=8.085746052164307, details={'was_impossible': False}),
 Prediction(uid='aleigh', iid=2904, r_ui=9.0, est=8.795337109450394, details={'was_impossible': False}),
 Prediction(uid='bigkfc2k8', iid=267, r_ui=6.0, est=8.749219580991028, details={'was_impossible': False}),
 Prediction(uid='Torah', iid=6956, r_ui=10.0, est=7.646700958814897, details={'was_impossible': False}),
 Prediction(uid='godT', iid=63, r_ui=7.0, est=9.035756771864333, details={'was_impossible': False}),
 Prediction(uid='lightofzeref', iid=36511, r_ui=7.0, est=6.818561045722257, details={'was_impossible': False}),
 Prediction(uid='ranmaSA', iid=401, r_ui=10.0, est=9.715147750644029, details={'was_impossible': False}),
 Prediction(uid='Josay', iid=16664, r_ui=9.0, est=7.39957469773722, details={'was_impossible': False}),
 Prediction(uid='Vanroy17', iid=58, r_ui=8.0, est=7.488599188049042, details={'was_impossible': False}),
 Prediction(uid='Ammes', iid=814, r_ui=9.0, 

In [10]:
recom_svd = algo_svd.predict(uid='Scieska', iid=58)
recom_svd

Prediction(uid='Scieska', iid=58, r_ui=None, est=6.7829371580230156, details={'was_impossible': False})

# KNN

In [11]:
sim_options = {'name': 'pearson_baseline',
               'shrinkage': 0  # no shrinkage
               }
algo_knn = KNNBasic(k=50)
prediction_knn = algo_knn.fit(trainset).test(testset)

# Prediksi
prediction_knn

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


[Prediction(uid='Shouichirou', iid=12679, r_ui=8.0, est=7.448923658126291, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid='aleigh', iid=2904, r_ui=9.0, est=9.157312132464861, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='bigkfc2k8', iid=267, r_ui=6.0, est=8.205612317426565, details={'actual_k': 31, 'was_impossible': False}),
 Prediction(uid='Torah', iid=6956, r_ui=10.0, est=7.685431315159591, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='godT', iid=63, r_ui=7.0, est=8.02858155077459, details={'actual_k': 48, 'was_impossible': False}),
 Prediction(uid='lightofzeref', iid=36511, r_ui=7.0, est=7.149786061188041, details={'actual_k': 9, 'was_impossible': False}),
 Prediction(uid='ranmaSA', iid=401, r_ui=10.0, est=9.119561451924637, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid='Josay', iid=16664, r_ui=9.0, est=8.28373986243068, details={'actual_k': 23, 'was_impossible': False}),
 Prediction(uid='Vanroy

In [12]:
recom_knn = algo_knn.predict(uid='Scieska', iid=58)
recom_knn

Prediction(uid='Scieska', iid=58, r_ui=None, est=7.257496793876683, details={'actual_k': 15, 'was_impossible': False})

# KNNBaseline

In [21]:
sim_options = {'name': 'pearson_baseline',
                'min_support': 1
               }
bsl_options =  {
        'method': 'als',
        'reg': 1
    }
algo_knn = KNNBaseline(k=40, sim_options=sim_options, bsl_options=bsl_options)
prediction_knn = algo_knn.fit(trainset).test(testset)

# Prediksi
prediction_knn

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


[Prediction(uid='Shouichirou', iid=12679, r_ui=8.0, est=7.466544675191275, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid='aleigh', iid=2904, r_ui=9.0, est=9.28604827595691, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='bigkfc2k8', iid=267, r_ui=6.0, est=8.736401918625553, details={'actual_k': 31, 'was_impossible': False}),
 Prediction(uid='Torah', iid=6956, r_ui=10.0, est=8.734453997258518, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='godT', iid=63, r_ui=7.0, est=8.273319474501767, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='lightofzeref', iid=36511, r_ui=7.0, est=7.127748926326364, details={'actual_k': 9, 'was_impossible': False}),
 Prediction(uid='ranmaSA', iid=401, r_ui=10.0, est=9.766685786174431, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid='Josay', iid=16664, r_ui=9.0, est=7.853661578456564, details={'actual_k': 23, 'was_impossible': False}),
 Prediction(uid='Vanro

# Testing Akurasi

In [16]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), KNNBasic(),SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNWithMeans(), KNNWithZScore(), BaselineOnly()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=5, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')  

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Don

,test_rmse,fit_time,test_time
Algorithm,,,
KNNBaseline,1.284677,1.881269,7.358095
SVD,1.297148,10.942441,0.968963
KNNWithZScore,1.297736,1.525951,7.376707
BaselineOnly,1.302467,0.642349,0.371392
KNNWithMeans,1.304767,1.300747,6.690079
SlopeOne,1.306776,5.036575,12.540886
KNNBasic,1.421043,1.209972,6.283666
KNNBasic,1.421654,1.220248,6.214731
NMF,2.133640,12.916737,0.375391


In [20]:
df = pd.DataFrame(prediction_knn)
df.to_csv("Prediksi_KNN_normal.csv")

df = pd.DataFrame(prediction_mf)
df.to_csv("Prediksi_SVD_normal.csv")